# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [1]:
import pandapower as pp
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_kw=60e3)
pp.create_load(net, bus3, p_kw=70e3)
pp.create_load(net, bus4, p_kw=10e3)

#create generators
eg = pp.create_ext_grid(net, bus1)
g0 = pp.create_gen(net, bus3, p_kw=-80*1e3, min_p_kw=0, max_p_kw=-80e3,vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_kw=-100*1e3, min_p_kw=0, max_p_kw=-100e3, vm_pu=1.01, controllable=True)

## Loss Minimization

We run an OPF:

In [2]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [3]:
net.res_ext_grid

,p_kw,q_kvar
0,-56530.142717,-1974.467822


In [4]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-71313.730159,1969.652493,-3.712797,1.000009
1,-12299.416890,1451.157410,-3.712788,1.000010


Since no individiual generation costs were specified, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [5]:
net.ext_grid.loc[eg, "cost_per_kw"] = 0.10
net.gen.loc[g0, "cost_per_kw"] = 0.15
net.gen.loc[g1, "cost_per_kw"] = 0.12

And now run an OPF:

In [6]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


We can see that all active power is provided by the external grid. This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [7]:
net.res_cost

14455.951448826132

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [8]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,144559.383879,9193.004654,-143959.455824,15993.781675,599.928055,25186.786329,0.380136,0.759988,144.851396


We now limit the transformer loading to 50%:

In [9]:
net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [10]:
pp.runopp(net)

We can see that the transformer complies with the maximum loading:

In [11]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,49953.831917,-2147.297933,-49833.781328,5167.433554,120.050589,3020.13562,0.131216,0.262153,49.999962


And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [12]:
net.res_ext_grid

,p_kw,q_kvar
0,-49953.831917,2147.297934


In [13]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-0.047352,-2993.136942,-6.232823,0.985230
1,-93304.066210,-3453.327899,-1.237880,1.025709


This comes of course with an increase in dispatch costs:

In [14]:
net.res_cost

16191.878239555741

### Line Loading Constraints

Wen now look at the line loadings:

In [15]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,19779.987854,-2479.429478,-19341.674584,1104.385359,438.313270,-1375.044119,0.104308,0.103207,0.104308,22.193296
1,-50658.278064,1888.751583,52783.674521,921.061618,2125.396457,2809.813201,0.270060,0.270140,0.270140,57.476525
2,30520.391688,2532.266282,-29946.206526,-2688.004075,574.185162,-155.737793,0.156712,0.157323,0.157323,33.473005


and run the OPF with a 50% loading constraint:

In [16]:
net.line["max_loading_percent"] = 50
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Now the line loading constraint is complied with:

In [17]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,16727.532318,-3194.198556,-16412.740583,1533.356250,314.791735,-1660.842305,0.088848,0.087130,0.088848,18.903796
1,-44451.211099,867.137901,46059.955230,832.560223,1608.744131,1699.698124,0.235000,0.235000,0.235000,49.999997
2,27533.096563,4051.891931,-27060.152701,-4429.882352,472.943862,-377.990422,0.141965,0.143058,0.143058,30.437792


And all generators are involved in supplying the loads:

In [18]:
net.res_ext_grid

,p_kw,q_kvar
0,-49787.594269,4603.789001


In [19]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-9136.048164,-2400.493544,-5.814910,0.992992
1,-83593.051621,-4884.451800,-1.511686,1.028900


This of course comes with a once again rising dispatch cost:

In [20]:
net.res_cost

16380.332845968116

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [21]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49787.594269,4603.789001
1,1.006025,-3.408832,60000.000000,0.000000
2,0.992992,-5.814910,60863.951836,-2400.493544
3,1.028900,-1.511686,-73593.051621,-4884.451800


and constrain it:

In [22]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net)

We can see that all voltages are within the voltage band:

In [23]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49906.847832,3050.617583
1,1.004168,-3.421015,60000.000000,0.000000
2,1.000000,-5.976094,59278.207273,-14858.927798
3,1.020000,-1.366892,-71863.493411,9172.685486


And all generators are once again involved in supplying the loads:

In [24]:
net.res_ext_grid

,p_kw,q_kvar
0,-49906.847832,3050.617583


In [25]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-10721.792727,-14858.927798,-5.976094,1.00
1,-81863.493411,9172.685486,-1.366892,1.02


This of course comes once again with rising dispatch costs:

In [26]:
net.res_cost

16422.572901622083

## DC OPF

pandapower also provides the possibility of running a DC Optimal Power Flow:

In [27]:
pp.rundcopp(net)

Since voltage magnitudes are not included in the DC power flow formulation, voltage constraints canot be considered in the DC OPF:

In [28]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,NaN,0.000000,-49999.999962,NaN
1,NaN,-3.436967,60000.000000,NaN
2,NaN,-5.708566,61488.746675,NaN
3,NaN,-1.362340,-71488.746713,NaN


Line and transformer loading limits are however complied with:

In [29]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,16715.233326,NaN,-16715.233326,NaN,NaN,NaN,0.087732,0.087732,0.087732,18.666430
1,-44773.513348,NaN,44773.513348,NaN,NaN,NaN,0.235000,0.235000,0.235000,50.000000
2,26715.233365,NaN,-26715.233365,NaN,NaN,NaN,0.140219,0.140219,0.140219,29.833747


In [30]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,49999.999962,NaN,-49999.999962,NaN,NaN,NaN,0.131216,0.262432,50.0


As are generator limits:

In [31]:
net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type,min_p_kw,max_p_kw,controllable,cost_per_kw
0,None,2,-80000.0,1.01,NaN,NaN,NaN,1.0,True,None,0.0,-80000.0,True,0.15
1,None,3,-100000.0,1.01,NaN,NaN,NaN,1.0,True,None,0.0,-100000.0,True,0.12


In [32]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-8511.253325,NaN,-5.708566,1.0
1,-81488.746713,NaN,-1.362340,1.0


The cost function is the same for the linearized OPF as for the non-linear one:

In [33]:
net.res_cost

16055.33760052851